In [1]:
!pip install -q sentence-transformers faiss-cpu transformers pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
import pdfplumber
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]  # Get uploaded file name


Saving Dummy_RAG_Research_Paper.pdf to Dummy_RAG_Research_Paper.pdf


In [4]:
def extract_text_from_pdf(pdf_path):
    all_text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            all_text += page.extract_text() + '\n'
    return all_text

document_text = extract_text_from_pdf(pdf_path)


In [5]:
def split_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(' '.join(chunk))
        i += chunk_size - overlap
    return chunks

chunks = split_text(document_text)


In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast model
embeddings = model.encode(chunks, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


In [8]:
def retrieve_top_k(query, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)
    return [chunks[i] for i in I[0]]


In [9]:
# You can try 'deepset/roberta-base-squad2' or other QA-capable models
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

def generate_answer(query):
    context = " ".join(retrieve_top_k(query))
    result = qa_pipeline(question=query, context=context)
    return result['answer']


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [11]:
question = "What are the main components of a RAG model?"
answer = generate_answer(question)
print("📌 Question:", question)
print("✅ Answer:", answer)


📌 Question: What are the main components of a RAG model?
✅ Answer: retrieval-based mechanisms with generative models like Transformers


In [12]:
# List of your custom questions
questions = [
    "What is Retrieval-Augmented Generation (RAG)?",
    "How does the retriever component work in RAG?",
    "What is the role of the generator in a RAG model?",
    "Why is vector similarity important in RAG?",
    "What models are commonly used in RAG pipelines?",
    "How does RAG use documents from a vector store?",
    "Can RAG be used without a language model like GPT?"
]

# Loop through questions and get answers
for q in questions:
    answer = generate_answer(q)
    print(f"📌 Question: {q}")
    print(f"✅ Answer: {answer}\n")


📌 Question: What is Retrieval-Augmented Generation (RAG)?
✅ Answer: Knowledge-Intensive NLP Tasks

📌 Question: How does the retriever component work in RAG?
✅ Answer: Finds top-k relevant chunks from the document store

📌 Question: What is the role of the generator in a RAG model?
✅ Answer: Generates answers conditioned on the retrieved context

📌 Question: Why is vector similarity important in RAG?
✅ Answer: bridges the gap

📌 Question: What models are commonly used in RAG pipelines?
✅ Answer: generative models

📌 Question: How does RAG use documents from a vector store?
✅ Answer: embeddings

📌 Question: Can RAG be used without a language model like GPT?
✅ Answer: bridges

